In [ ]:
import pandas as pd
import re
import emoji
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

taglish_stopwords = set(stopwords.words('english')) | {
    'ako','ikaw','siya','tayo','kami','kayo','sila','ang','ng','sa','na','ay','ko',
    'mo','ni','ito','ganyan','ganun','lang','naman','pa','rin','ba','ha','eh','o',
    'nga','daw','din','pala','natin','nyo','sayo','yan','wala','meron'
}

tagalog_slang_dict = {
    "nmn": "naman", "pki": "paki", "pls": "please", "bkit": "bakit",
    "d": "hindi", "di": "hindi", "nde": "hindi", "kaau": "masyado",
    "paun": "paano", "kc": "kasi", "q": "ko", "u": "ikaw", "ung": "iyong",
    "kya": "kaya", "tlga": "talaga", "anu": "ano", "dn": "din", "mg": "mga",
    "na2": "na ito", "sya": "siya", "skl": "share ko lang", "grabeh": "grabe",
    "pkisagot": "pakisagot", "gnyan": "ganyan", "nyo": "ninyo"
}

def normalize_tagalog_words(tokens):
    return [tagalog_slang_dict.get(token, token) for token in tokens]

def clean_taglish_xpost(text):
    text = str(text).lower()
    text = emoji.replace_emoji(text, replace="")
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)

    tokens = text.split()

    tokens = normalize_tagalog_words(tokens)

    laughter_regex = re.compile(r'\b(ha)+\b|\b(he)+\b|\b(ho)+\b|\blol\b', re.IGNORECASE)
    filtered = [word for word in tokens if word not in taglish_stopwords or laughter_regex.match(word)]

    return " ".join(filtered)

df = pd.read_csv("for_export_philippine_elections.csv")  
df["cleaned"] = df["text"].apply(clean_taglish_xpost)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Okaru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df = df[df["cleaned"].str.strip() != ""]

df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df_1000 = df.head(1000)

df_1000["label"] = ""

df_1000[["cleaned", "label"]].to_csv("taglish_1000_for_annotation.csv", index=False)

print("1000 cleaned tweets exported to 'taglish_1000_for_annotation.csv'")

✅ Done! 1000 cleaned tweets exported to 'taglish_1000_for_annotation.csv'


C:\Users\Okaru\AppData\Local\Temp\ipykernel_1916\3589578770.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1000["label"] = ""
